In [1]:
#Импортируем необходимые библиотеки

import pandas as pd
import math
import numpy as np
import scipy as sp
import datetime

import statsmodels as st
import statsmodels.api as sm
from statsmodels.stats import power

In [2]:
#Задаем параметры

#Массив со значениями MDE для каждой метрики
MDEs=[4.05, 3.03]

#Список названий метрик
metrics_names=["CR_1%", "CR_2%"]

#Список булевых значений-ответов на вопрос "Метрика-конверсия?"
metrics_bool=[1, 1]

test_alpha=0.05
test_power=0.8

#Дата начала АВ-теста в формате "YYYY-mm-dd"
start_date_test='2024-01-16'
end_date_hist=(datetime.datetime.strptime(start_date_test, '%Y-%m-%d').date()+datetime.timedelta(days=-1)).strftime('%Y-%m-%d')
start_date_hist=(datetime.datetime.strptime(start_date_test, '%Y-%m-%d').date()+datetime.timedelta(days=-31)).strftime('%Y-%m-%d')

In [3]:
#Эти переменные понадобятся в дальнейшем, но в них мы ничего руками не вписываем

#Дата конца А/В-теста, мы здесь как раз подсчитаем
end_date_test=""

#Количество метрик
metrics_num=len(MDEs)

In [4]:
#Считываем данные из csv-фалйа
df_raw_hist=pd.read_csv("df_raw_hist_length.csv")
df_raw_hist.set_index('event_date', inplace=True)

In [5]:
#Загрузили исторические данные по метрикам в датафрейм
df_hist=df_raw_hist.copy()
df_hist=df_hist.iloc[:, :metrics_num]

#Загрузили исторические данные о среднем числе пользователей в день в датафрейм
df_hist_freq=df_raw_hist.copy()
df_hist_freq=df_hist_freq.iloc[:, metrics_num:]

df_hist_freq=[df_hist_freq.iloc[:, i].mean() for i in range(metrics_num)]
df_hist_freq

[418.64516129032256, 418.64516129032256]

In [6]:
#Создаем датафрейм для информации о требуемой длительности теста по каждой метрике
metrics_info_df=pd.DataFrame()
#Итоговая длительность А/В-теста
days_res=-1
#Количество человек в 1 выборке
sample_size_res=-1
#Среднее количество пользователей в день
mean_freq_res=df_hist_freq[0]
#Для каждой из метрик проводим алгоритм:
for i in range(metrics_num):
    #Считываем среднее количество пользователей по ней
    mean_freq=df_hist_freq[i]
    #Рассчитываем ее среднее на исторических данных
    mean_metric=df_hist.iloc[:, i].values.mean()
    #Рассчитываем ее стандартное отклонение на исторических данных
    std_metric=df_hist.iloc[:, i].values.std()
    #Если метрика-конверсия, обновляем стандартное отклонение
    if metrics_bool[i]:
        std_metric=max(std_metric, math.sqrt(mean_metric*(100-mean_metric)))
    #Рассчитываем относительный MDE
    MDE_rel=MDEs[i]/std_metric
    #Рассчитываем величину одной выборки
    sample_size_curr=power.tt_ind_solve_power(effect_size=MDE_rel, nobs1=None, alpha=test_alpha, 
        power=test_power, ratio=1.0, alternative='two-sided')
    #Рассчитываем продолжительность теста для данной метрики
    days_curr=math.ceil(sample_size_curr*2/mean_freq)
    #Заносим данные по данной метрике в общий датафрейм
    curr_metric_df=pd.DataFrame(data={"Метрика": [metrics_names[i]], 
                                      "Необходимый размер одной выборки (чел.)": [math.ceil(sample_size_curr)], 
                                      "Продолжительность теста (в днях)": [days_curr], 
                                      "Средний трафик в день (чел.)": [round(mean_freq)]})
    metrics_info_df=pd.concat([metrics_info_df, curr_metric_df])
    #Выбираем метрику с самой большой рассчитанной продолжительностью и сохраняем данные по ней
    if days_curr>days_res:
        days_res=days_curr
        sample_size_res=sample_size_curr
        mean_freq_res=mean_freq  
        
#Сортируем датафрейм по убыванию продолжительности теста в днях
metrics_info_df.sort_values(by="Продолжительность теста (в днях)", inplace=True, ascending=False)
#Вычисляем дату конца проведения А/В-теста
end_date_test=(datetime.datetime.strptime(start_date_test, '%Y-%m-%d').date()+datetime.timedelta(days=days_res-1)).strftime('%Y-%m-%d')
#Выводим количесвто человек, необходимое для теста, итоговый средний трафик в день и итоговую продолжительность теста в днях
print(f"Для проведени А/В-теста необходимо {math.ceil(sample_size_res)} пользователей в одной группе. Среднее число пользователей в день составляет {round(mean_freq_res)} человек, значит, для А/В-теста понадобится {days_res} дней")
print(f"Тогда данный А/В-тест будет идти с {start_date_test} по {end_date_test}")        

Для проведени А/В-теста необходимо 2096 пользователей в одной группе. Среднее число пользователей в день составляет 419 человек, значит, для А/В-теста понадобится 11 дней
Тогда данный А/В-тест будет идти с 2024-01-16 по 2024-01-26


In [7]:
#Распечатали датафрейм с информацией о длительности теста для каждой метрики
metrics_info_df

,Метрика,Необходимый размер одной выборки (чел.),Продолжительность теста (в днях),Средний трафик в день (чел.)
0,CR_1%,2096,11,419
0,CR_2%,1833,9,419


In [8]:
#Формируем список продолжительности АВ-теста в днях для каждой метрики и КОПИРУЕМ ВЫВОД жанной ячейки в ноутбук "Подсчеты и итоги"
lengths_list=list(metrics_info_df["Продолжительность теста (в днях)"].values)
lengths_list

[11, 9]